## rush parallel - windows only

In [1]:
import sys,os,shutil,subprocess
import glob,tempfile
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
import matplotlib as mpl
sys.path.append('c:/Users/damien/projects/snipgenie/')
from snipgenie import tools, aligners, app, trees, plotting

In [5]:
reload(app)
reload(tools)
args = {'threads':4, 'outdir': "E:/test_results", 'labelsep':'_',
        'input':['E:/test_files'],        
         'species': 'Mbovis-AF212297','get_stats':False,
         'overwrite':True,
         'aligner':'minimap2',
         'custom_filters': True,
         'buildtree':True}
W = app.WorkFlow(**args)
st = W.setup()
W.run()

The following options were supplied
-------
threads : 4
outdir : E:/test_results
labelsep : _
input : ['E:/test_files']
species : Mbovis-AF212297
get_stats : False
overwrite : True
aligner : minimap2
custom_filters : True
buildtree : True
labelindex : 0
trim : False
unmapped : False
quality : 25
platform : illumina
filters : QUAL>=40 && FORMAT/DP>=30 && DP4>=4
mask : None
reference : None
gb_file : None
omit_samples : []
bootstraps : 100

4 samples were loaded:
----------------------
     sample                                          filename1                                          filename2                       name1                       name2  read_length
0  13-11594  E:\test_files\13-11594_S85_L001-4_R1_001.fastq.gz  E:\test_files\13-11594_S85_L001-4_R2_001.fastq.gz  13-11594_S85_L001-4_R1_001  13-11594_S85_L001-4_R2_001          149
1  15-11643  E:\test_files\15-11643_S67_L001-4_R1_001.fastq.gz  E:\test_files\15-11643_S67_L001-4_R2_001.fastq.gz  15-11643_S67_L001-4_R1_001  15

In [ ]:
trees.run_fasttree('E:/test_results/core.fa', bootstraps=100, outpath='E:/test_results/')

In [19]:
bams=glob.glob('E:/test_results/mapped/*.bam')
bam_files = ' '.join(bams[:2])
print (bam_files)

E:/test_results/mapped\15-11643.bam E:/test_results/mapped\13-11594.bam


In [ ]:
import re
re.findall( '[^:]*$','"LT708304.1":1087-2174')

['1087-2174', '']

In [120]:
tempdir = tempfile.tempdir
regstr = '"LT708304.1":1-1087 "LT708304.1":1087-2174'
filesstr = "1.bcf 2.bcf"
annotatestr = '"AD,ADF,ADR,DP,SP,INFO/AD,INFO/ADF,INFO/ADR"'
outfiles = [r"\Users\damien\AppData\Local\Temp\1-1087.bcf",r"C:\Users\damien\AppData\Local\Temp\1087-2174.bcf"]
rawbcf = 'raw.bcf'

ref=  app.mbovis_genome
rushcmd = tools.get_cmd('rush')
bcftoolscmd = tools.get_cmd('bcftools')
cmd = 'echo {reg} | {rc} -D " " "{bc} mpileup -r {{}} -f {r} -a {a} --min-MQ 60 {b} -o {p}/{{@[^:]*$}}.bcf"'\
        .format(rc=rushcmd,bc=bcftoolscmd,reg=regstr,r=ref,b=bam_files,a=annotatestr,p=tempdir)
print (cmd)
tmp=subprocess.check_output(cmd, shell=True)
cmd = '{bc} concat {i} -O b -o {o}'.format(bc=bcftoolscmd,i=' '.join(outfiles),o=rawbcf)
tmp=subprocess.check_output(cmd, shell=True)


echo "LT708304.1":1-1087 "LT708304.1":1087-2174 | C:\Users\damien\.config\snipgenie\binaries\rush.exe -D " " "C:\Users\damien\.config\snipgenie\binaries\bcftools.exe mpileup -r {} -f C:\Users\damien\.config\snipgenie\genome\Mbovis_AF212297.fa -a "AD,ADF,ADR,DP,SP,INFO/AD,INFO/ADF,INFO/ADR" --min-MQ 60 E:/test_results/mapped\15-11643.bam E:/test_results/mapped\13-11594.bam -o C:\Users\damien\AppData\Local\Temp/{@[^:]*$}.bcf"


In [129]:
reload(app)
bams = app.get_files_from_paths('E:/test_results/mapped/','*.bam')
print (bams)
app.mpileup_parallel(bams, app.mbovis_genome, '.' )

['E:/test_results/mapped\\15-11643.bam', 'E:/test_results/mapped\\13-11594.bam', 'E:/test_results/mapped\\19-11957.bam', 'E:/test_results/mapped\\17-11662.bam']
[      1 1087476 2174952 3262428 4349904]
echo "LT708304.1":1-1087475 "LT708304.1":1087476-2174951 "LT708304.1":2174952-3262427 "LT708304.1":3262428-4349903 | C:\Users\damien\.config\snipgenie\binaries\rush.exe -D " " "C:\Users\damien\.config\snipgenie\binaries\bcftools.exe mpileup -r {} -f C:\Users\damien\.config\snipgenie\genome\Mbovis_AF212297.fa -a "AD,ADF,ADR,DP,SP,INFO/AD,INFO/ADF,INFO/ADR" --min-MQ 60 E:/test_results/mapped\15-11643.bam E:/test_results/mapped\13-11594.bam E:/test_results/mapped\19-11957.bam E:/test_results/mapped\17-11662.bam -o C:\Users\damien\AppData\Local\Temp/{@[^:]*$}.bcf"
C:\Users\damien\.config\snipgenie\binaries\bcftools.exe concat C:\Users\damien\AppData\Local\Temp\1-1087475.bcf C:\Users\damien\AppData\Local\Temp\1087476-2174951.bcf C:\Users\damien\AppData\Local\Temp\2174952-3262427.bcf C:\Users

'.\\raw.bcf'